In [ ]:
import ipywidgets as widgets
import ipyvue as vue
from traitlets import (Dict, Unicode, List, Instance, observe)

In [ ]:
mydiv = vue.Html(tag='div', children=['mydiv'])

In [ ]:
class CoolComponent(vue.VueTemplate):
    input = Unicode().tag(sync=True)
    something = Unicode('defaultvalue').tag(sync=True)
    template = Unicode('''
        <div style="border: 1px solid green; margin: 2px" @click="append_one()">
            [{{ something }}] input: {{ input }} 
        </div>
    ''').tag(sync=True)
    
    def vue_append_one(self, *args):
        self.input = f'{self.input}1'
    
CoolComponent(input='some text')    

In [ ]:
class Cls(vue.VueTemplate):
    texts = List(['xxxx', 'yyyy']).tag(sync=True)
    direct = Unicode('aaa').tag(sync=True)
    template = Unicode('''
        <div>
            <div style="border: 1px solid black">
               <cool-component v-for="t in texts" :input="t" :key="t" />
            </div>
            <cool-component :input="direct" something="---"/>
            <w-button v-for="t in texts" :description="t" :key="t" />
            <w-button description="no v-for"/>
            <some-instance />
        </div>
    ''').tag(sync=True)

cls = Cls(components={
    'cool-component': CoolComponent,
    'some-instance': mydiv,
    'w-button': widgets.Button})   

cls

In [ ]:
cls.texts=['xxxx', 'zzzz']

In [ ]:
cls._component_instances

In [ ]:
cls.direct = 'bbb'

# Non serializable properties

In [ ]:
class Database():
    def __init__(self, url):
        self.url = url

class DatabaseInfo(vue.VueTemplate):
    db = Instance(Database)
    info = Unicode().tag(sync=True)
    label = Unicode().tag(sync=True)
    
    template = Unicode('''
        <div style="border: 1px solid green; margin: 2px">
            [{{ label }}] Db URL: {{ info }}
        </div>
    ''').tag(sync=True)
    
    @observe('db')
    def db_changed(self, change):
        self.info = self.db.url

In [ ]:
class MyApp(vue.VueTemplate):
    
    customer_db = Instance(Database).tag(sync_ref=True)
    supplier_db_collection = Dict().tag(sync_ref=True)
    
    template = Unicode('''
        <div>
            <database-info label="customers" :db="customer_db" />
            
            <database-info
                label="supplier"
                v-for="supplier in supplier_db_collection.preferred"
                :db="supplier"
                :key="supplier.id" />
            
            <database-info label="function" :db="{functionRef: 'db_factory', args: ['localhost/function1']}" />
        </div>
    ''').tag(sync=True)
    
    components = Dict({
        'database-info': DatabaseInfo
    }).tag(sync=True, **vue.VueTemplate.class_component_serialization)
    
    def db_factory(self, url):
        return Database(url)

my_app = MyApp(
    customer_db = Database('localhost/customers'),
    supplier_db_collection = {'preferred': [Database('localhost/intel')]}
)
my_app   

In [ ]:
my_app.customer_db = Database('remote/customers_v2')

In [ ]:
my_app.supplier_db_collection = {'preferred': [Database('remote/amd')]}